# Plugin Manager

In [ ]:
import os
import glob
import pydash
import json
import naas_python

# Create Chat commands from directory
def create_command(input_dir):
    # Init
    commands = []
    connectors = []
    files = sorted(glob.glob(os.path.join(input_dir, "**", "*.ipynb"), recursive=True) + connectors, key=lambda x: x.split("/")[-1])

    # Loop on .ipynb files
    for file in files:
        # Init
        command = {}
        section = file.split("/")[-2]
        file_name = file.split("/")[-1]

        # Exclude file name chat plugin or pipeline
        if not file_name.startswith("__") and not file_name.endswith("__") and not "utils" in file and not '/home/ftp/.naas/' in file:
            # Open file and get cells
            with open(file) as f:
                nb = json.load(f)
            cells = nb.get("cells")

            # Find tags #command to generate command
            is_command = False
            for cell in cells:
                source = cell.get("source")
                if "#command" in " ".join(source):
                    is_command = True
                    break

            if is_command:
                # Get name & webhook
                payload = {}
                command["name"] = file_name.replace(".ipynb", "")
                command["avatar"] = cells[0].get("source")[0].split('src="')[1].split('"')[0]
                command["action"] =  {
                    "request_type": "POST",
                    "url": naas.webhook.add(file, params={"inline": True}),
                    "payload": {}
                }
                payload = {}

                # Add command description & payload
                for i, cell in enumerate(cells):
                    source = "".join(cell.get("source"))
                    tags = pydash.get(cell, "metadata.tags")
                    # Create description
                    if "**Description:**" in source:
                        command_description = source.split("**Description:**")[1].strip()
                        command["description"] = command_description.replace('notebook', 'command')
                    # Create command payload
                    if tags and "variables" in tags or "Setup variables" in source:
                        variables = source
                        defaults = "".join(cells[i+1].get("source"))
                    if tags and "parameters" in tags:
                        for s in cells[i+1].get("source"):
                            if "=" in s:
                                parameter = s.split("=")[0].strip()
                                payload[parameter] = {
                                    "type": "str",
                                    "description": variables.split(f"`{parameter}`:")[-1].split("-")[0].replace("\n", "").strip(),
                                    "default": defaults.split(f"{parameter} =")[-1].split("\n")[0].strip(),
                                }
                command["action"]["payload"] = payload

                # Generate commands (to be develop)
                commands.append(command)
    return commands

# Create plugin
def create_plugin(
    name=None,
    prompt="",
    model="gpt-3.5-turbo-16k",
    temperature=0,
    output_dir=None,
    output_path=None,
    model_dir=None,
    description="",
    avatar="",
    prompt_type="system",
    entity_name=None,
    slug=None,
):
    # Init
    commands = []
    plugin_url = None
    plugin = {}
    
    # Create output path
    if not output_path:
        output_path = remove_emojis(os.path.join(output_dir, f"{name.lower().replace(' ', '_')}.json"))
        
    # Create commands
#     print("--> Creating commands")
#     commands = create_command(model_dir)
#     print()
        
    # Create empty plugin
    if not os.path.exists(output_path):
        with open(output_path, "w") as f:
            json.dump(plugin, f)
        am.add(output_path)
        
    # Get asset URL
    plugin_url = am.get_asset_url(output_path)
    
    # Check tokens
    prompt_tokens, max_tokens = naas_chat_plugin.check_tokens(prompt, model)

    # Create JSON
    entity_id = entity_name.lower().replace(' ', '-')
    plugin_id = f"{name.lower().replace(' ', '-')}-{entity_id}"
    plugin_name = f"{name} - {entity_name}"
    plugin_slug = f"{name.lower().replace(' ', '-')}/{entity_id}"
    if slug is not None:
        plugin_slug = slug
        plugin_name = name
    plugin = {
        "id": plugin_id,
        "slug": plugin_slug,
        "url": plugin_url,
        "name": plugin_name,
        "model": model,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "prompt": prompt,
        "commands": commands,
        "description": description,
        "avatar": avatar,
        "prompt_type": prompt_type,
    }

    # Save dict to JSON file
    with open(output_path, "w") as f:
        json.dump(plugin, f)
    print(f"💾 Plugin successfully saved. You can use it in your Naas Chat with: {output_path}")
    
    # Upload file to storage manager
    sm.upload_file(output_path)
    return output_path, plugin_url, plugin

# Push plugin to workspaces
def push_plugin_to_workspace(
    api_key,
    plugin,
    workspace_ids=[],
    personal_workspace=True
):
    # Init
    result = None
    if isinstance(workspace_ids, str) and workspace_ids == '':
        workspace_ids = []
        
    # Get workspaces
    workspaces = list_workspaces(api_key)
    
    # Get existing workspace ids
    current_workspace_ids = [workspace.get("id") for workspace in workspaces.get("workspaces")]
    
    # Add personal workspace
    personal_workspace_id = get_personal_workspace(api_key)
    if personal_workspace and personal_workspace_id not in workspace_ids:
        workspace_ids.append(personal_workspace_id)
    
    # Push plugin
    for index, workspace_id in enumerate(workspace_ids):
        workspace_id = workspace_id.strip()
        if workspace_id in current_workspace_ids:
            print(f"{index} - Workspace ID:", workspace_id)
            
            # List existing plugins
            plugins = list_workspace_plugins(
                api_key,
                workspace_id,
            )
            
            # Create, update, delete plugin
            plugin_exist = False
            current_plugin_id = None
            plugin_slug = plugin.get("slug")
            plugin_name = plugin.get("name")
            plugin_url = plugin.get("url")
            print("Plugins:", len(plugins.get('workspace_plugins')))
            for i, p in enumerate(plugins.get('workspace_plugins')):
                plugin_id = p.get("id")
                p_json = json.loads(p.get("payload"))
                p_name = p_json.get("name")
                p_slug = p_json.get("slug")
                p_url = p_json.get("url")
                if plugin_slug == p_slug and plugin_url != p_url:
                    res = delete_workspace_plugin(api_key, workspace_id, plugin_id)
                    print(f"🗑️ Plugin '{p_name}' successfully deleted {plugin_id}")
                elif plugin_slug == p_slug and plugin_url == p_url:
                    plugin_exist = True
                    current_plugin_id = plugin_id
                    
            if not plugin_exist:
                result = create_workspace_plugin(
                    api_key,
                    workspace_id,
                    plugin,
                )
                print(f"✅ Plugin '{plugin_name}' successfully created.")
            else:
                result = update_workspace_plugin(
                    api_key,
                    workspace_id,
                    current_plugin_id,
                    plugin,
                )
                print(f"✅ Plugin '{plugin_name}' successfully updated.")
            print()
        else:
            print("❌ Workspace ID does not exist for user.")
    return result